In [1]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
import hgtk
plt.style.use('seaborn')
sns.set_palette("hls")
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = 'Malgun Gothic'

In [2]:
train_df = pd.read_json('../train_df(filtered_title(특수기호 안 없앰)).json')
train_df

,tags,id,plylst_title,songs,like_cnt,updt_date,tokenized_title,title_tags
0,"[힐링, 휴식, 밤, 새벽]",147668,to. 힘들고 지친 분들에게,"[663185, 649626, 6855, 188486, 348451, 169945,...",12,2016-06-23 10:06:27.000,"[힘들다, 지치다, 분들]","[힐링, 휴식, 밤, 새벽, 힘들다, 지치다, 분들]"
1,[팝],50422,130807-7,"[627035, 256438, 603324, 200889, 441319, 21689...",0,2013-08-15 13:17:11.000,[130807-7],"[팝, 130807-7]"
2,[뉴에이지],116432,숙면을 위한 슬픈 마음을 달래 줄 피아노,"[129204, 369497, 649743, 344619, 110281, 63266...",23,2015-09-03 16:51:50.000,"[숙면, 슬프다, 마음, 달래, 피아노]","[뉴에이지, 숙면, 슬프다, 마음, 달래, 피아노]"
3,"[하드락, 록스피릿, 댄스]",55076,당신을 하얗게 불태울 곡들,"[677591, 420396, 104934, 119279, 251988, 58850...",1,2017-01-09 15:41:25.000,"[당신, 하얗다, 불, 태우다, 곡]","[하드락, 록스피릿, 댄스, 당신, 하얗다, 불, 태우다, 곡]"
4,"[힐링, 휴식, 기분전환]",125064,[스피커 필수 / hiphop] 듣고 있음 꿀렁꿀렁이고 싶은 힙합음악!,"[704455, 694036, 508043, 154933, 57614, 645195...",715,2016-02-22 12:32:50.000,"[스피커, 필수, hiphop, 듣다, 꿀렁꿀렁, 싶다, 힙합, 음악, !]","[힐링, 휴식, 기분전환, 스피커, 필수, hiphop, 듣다, 꿀렁꿀렁, 싶다, ..."
...,...,...,...,...,...,...,...,...
115066,[],132039,이것만 알아도 브릿팝 안다는 소리듣는다.,"[8778, 5725, 234307, 64368, 691112, 581349, 40...",232,2014-03-10 19:08:16.000,"[브릿팝, 소리, 듣다]","[브릿팝, 소리, 듣다]"
115067,[],21475,make015,"[470489, 72225, 387567, 328452, 517417, 690761...",0,2016-05-16 15:34:18.000,"[make, 015]","[make, 015]"
115068,"[힘들때, 지칠때, 힘내]",81196,"힘든 걸 알아, 말해주고 싶었어요","[448250, 131741, 531820, 616260, 639943, 13129...",13,2017-12-20 16:32:07.000,"[힘드다, 말, 해주, 고, 싶다]","[힘들때, 지칠때, 힘내, 힘드다, 말, 해주, 고, 싶다]"
115069,[],151600,♥christmas music♥,[],8,2015-12-23 09:38:19.000,"[♥, christmas, music, ♥]","[♥, christmas, music, ♥]"


In [3]:
def word_to_jamo(token):
    def to_special_token(jamo):
        if not jamo:
            return '-'
        else:
            return jamo
    
    decomposed_token = ''
    for char in token:
        try:
            # char(음절)을 초성, 중성, 종성으로 분리
            cho, jung, jong = hgtk.letter.decompose(char)
            
            # 자모가 빈 문자일 경우 특수문자 -로 대체
            cho = to_special_token(cho)
            jung = to_special_token(jung)
            jong = to_special_token(jong)
            decomposed_token = decomposed_token + cho + jung + jong
        
        # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                decomposed_token += char
    # 단어 토큰의 자모 단위 분리 결과를 추가
    return decomposed_token

In [4]:
word_to_jamo('여동생')

'ㅇㅕ-ㄷㅗㅇㅅㅐㅇ'

In [5]:
def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in s]

In [6]:
train_df['title_tags'][0]

['힐링', '휴식', '밤', '새벽', '힘들다', '지치다', '분들']

In [7]:
tokenize_by_jamo(train_df['title_tags'][0])

['ㅎㅣㄹㄹㅣㅇ', 'ㅎㅠ-ㅅㅣㄱ', 'ㅂㅏㅁ', 'ㅅㅐ-ㅂㅕㄱ', 'ㅎㅣㅁㄷㅡㄹㄷㅏ-', 'ㅈㅣ-ㅊㅣ-ㄷㅏ-', 'ㅂㅜㄴㄷㅡㄹ']

In [8]:
tokenized_data = []

for sample in tqdm(train_df['title_tags'].tolist()):
    tokenized_sample = tokenize_by_jamo(sample)
    tokenized_data.append(tokenized_sample)
tokenized_data

100%|███████████████████████████████████████████████████████████████████████| 115071/115071 [00:06<00:00, 18360.43it/s]


[['ㅎㅣㄹㄹㅣㅇ', 'ㅎㅠ-ㅅㅣㄱ', 'ㅂㅏㅁ', 'ㅅㅐ-ㅂㅕㄱ', 'ㅎㅣㅁㄷㅡㄹㄷㅏ-', 'ㅈㅣ-ㅊㅣ-ㄷㅏ-', 'ㅂㅜㄴㄷㅡㄹ'],
 ['ㅍㅏㅂ', '130807-7'],
 ['ㄴㅠ-ㅇㅔ-ㅇㅣ-ㅈㅣ-', 'ㅅㅜㄱㅁㅕㄴ', 'ㅅㅡㄹㅍㅡ-ㄷㅏ-', 'ㅁㅏ-ㅇㅡㅁ', 'ㄷㅏㄹㄹㅐ-', 'ㅍㅣ-ㅇㅏ-ㄴㅗ-'],
 ['ㅎㅏ-ㄷㅡ-ㄹㅏㄱ',
  'ㄹㅗㄱㅅㅡ-ㅍㅣ-ㄹㅣㅅ',
  'ㄷㅐㄴㅅㅡ-',
  'ㄷㅏㅇㅅㅣㄴ',
  'ㅎㅏ-ㅇㅑㅎㄷㅏ-',
  'ㅂㅜㄹ',
  'ㅌㅐ-ㅇㅜ-ㄷㅏ-',
  'ㄱㅗㄱ'],
 ['ㅎㅣㄹㄹㅣㅇ',
  'ㅎㅠ-ㅅㅣㄱ',
  'ㄱㅣ-ㅂㅜㄴㅈㅓㄴㅎㅘㄴ',
  'ㅅㅡ-ㅍㅣ-ㅋㅓ-',
  'ㅍㅣㄹㅅㅜ-',
  'hiphop',
  'ㄷㅡㄷㄷㅏ-',
  'ㄲㅜㄹㄹㅓㅇㄲㅜㄹㄹㅓㅇ',
  'ㅅㅣㅍㄷㅏ-',
  'ㅎㅣㅂㅎㅏㅂ',
  'ㅇㅡㅁㅇㅏㄱ',
  '!'],
 ['ㅇㅛ-ㅈㅡㅁㄷㅡㄷㄴㅡㄴㄴㅗ-ㄹㅐ-', 'ㅇㅛ-ㅈㅡㅁ', 'ㄷㅡㄷㄷㅏ-', 'ㄴㅗ-ㄹㅐ-', '2'],
 ['ㅋㅏ-ㅍㅔ-', 'ㅇㅣㄴㄷㅣ-', 'tthing', 'ㅎㅏ-ㄹㅜ-'],
 ['ㄹㅗ-ㅇㅜ-ㅍㅏ-ㅇㅣ-',
  'ㄱㅏㅁㅅㅓㅇ',
  'ㅈㅏ-ㄱㅣ-ㅈㅓㄴㅇㅔ-ㄷㅡㄷㄱㅣ-ㅈㅗㅎㅇㅡㄴ',
  'ㅇㅏㄹㅇㅐㄴㅂㅣ-ㅎㅣㅂㅎㅏㅂ',
  'ㅂㅜㄴㅇㅟ-ㄱㅣ-',
  'lofi',
  'rnb',
  'ㅁㅗ-ㅇㅡㅁ'],
 ['ㄷㅐㄴㅅㅡ-',
  'EDM',
  'ㄱㅏ-ㅇㅡㄹ',
  'ㅎㅏ-ㄴㅡㄹ',
  'edm',
  'ㅎㅏㅁㄲㅔ-',
  '!',
  'ㅇㅣㄹ',
  'ㄹㅔㄱ',
  'ㅌㅡ-ㄹㅣㄱ',
  'ㅅㅡ-ㅋㅏ-ㅇㅣ-',
  'ㅁㅠ-ㅈㅣㄱ',
  'ㅍㅔ-ㅅㅡ-ㅌㅣ-ㅂㅓㄹ',
  'ㅍㅡ-ㄹㅣ-ㅂㅠ-'],
 ['ㅂㅏㄹㄹㅏ-ㄷㅡ-', 'ㅂㅏㄹㄹㅏ-ㄷㅡ-', 'ㄱㅏㅁㅅㅓㅇ', 'ㅈㅐ-'],
 ['ㅎㅠ-ㅅㅣㄱ',
  'ㅎㅣㄹㄹㅣㅇ',
  'ㅇㅕ-ㅎㅐㅇ',
  'ㅅㅏㄴㅊㅐㄱ',
  '♤',
  'ㄱㅕ-ㅇㅜㄹ',
  'ㅇㅣ-ㅁㅕㄴ',
  'ㅅㅐㅇㄱㅏㄱ',
  'ㄴㅏ-',
  'ㄴㅡㄹㄷㅏ-',
  'ㄷㅡㄹㄹㅕ-ㅇㅗ-ㄷ

In [9]:
# 초성, 중성, 종성을 입력받으면 역으로 단어로 바꿔주는 jamo_to_word 함수를 구현
def jamo_to_word(jamo_sequence):
    tokenized_jamo = []
    index = 0
    
    # 1. 초기 입력
    # jamo_sequence = ' ㄴ ㅏ ㅁ ㄷ ㅗ ㅇ ㅅ ㅐ ㅇ '
    
    while index < len(jamo_sequence): # 각 문자에 대해서 세 개씩 분리하여 초성, 중성, 종성을 하나의 묶음으로 간주함
        # 문자가 한글(정상적인 자모)이 아닐 경우
        if not hgtk.checker.is_hangul(jamo_sequence[index]):
            tokenized_jamo.append(jamo_sequence[index])
            index = index + 1
        
        # 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주.
        else:
            tokenized_jamo.append(jamo_sequence[index:index + 3])
            index = index + 3
            
    # 2. 자모 단위 토큰화 완료
    # tokenized_jamo : [' ㄴ ㅏ ㅁ ', ' ㄷ ㅗ ㅇ ', ' ㅅ ㅐ ㅇ ']
    
    word = ''
    try:
        for jamo in tokenized_jamo:
            
            # 초성, 중성, 종성의 묶음으로 추정되는 경우
            if len(jamo) == 3:
                if jamo[2] == "-":
                    # 종성이 존재하지 않는 경우
                    word = word + hgtk.letter.compose(jamo[0], jamo[1])
                else:
                    # 종성 이 존재 하는 경우
                    word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
            else:
                word = word + jamo
    # 복원 중 (hgtk.letter.compose) 에러 발생 시 초기 입력 리턴.
    # 복원이 불가능한 경우 예시) 'ㄴ ! ㅁ ㄷ ㅗ ㅇ ㅅ ㅐ ㅇ '
    except Exception as exception:
        if type(exception).__name__ == 'NotHangulException':
            return jamo_sequence
    # 3. 단어로 복원 완료
    # word : '남 동 생'
    
    return word

In [10]:
jamo_to_word('ㄴㅏㅁㄷㅗㅇㅅㅐㅇ')

'남동생'

In [11]:
jamo_to_word('Rock')

'Rock'

In [12]:
import fasttext

In [13]:
with open('tokenized_data.txt', 'w', encoding='utf-8') as out:
    for line in tqdm(tokenized_data, unit='line'):
        out.write(' '.join(line) + '\n')

100%|████████████████████████████████████████████████████████████████████| 115071/115071 [00:00<00:00, 684736.60line/s]


In [14]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='skipgram',
                                    dim = 100,
                                    ws=30,
                                    epoch=10,
                                    minCount=1,
                                    thread=4,
                                    loss='ns',
                                    neg=50,
                                    verbose=2
                                   )
model.save_model("title2vec(fasttext_jamo).bin")

In [15]:
model = fasttext.load_model("title2vec(fasttext_jamo).bin")

In [16]:
model[word_to_jamo('힐링')]

array([ 0.4414031 , -0.01611445, -0.11355736,  0.41410524,  0.39524013,
       -0.7019967 ,  0.4800302 , -0.33659768,  0.46150294,  0.4676073 ,
        0.06843347, -0.3183036 ,  0.5029162 , -0.10788705, -0.08941298,
        0.31765145, -0.20978872, -0.5187679 , -0.23706092, -0.43003345,
        0.02573475, -0.29397127, -0.24411859,  0.07185571, -0.5033345 ,
       -0.15384468,  0.17378885,  0.20414023,  1.1999539 , -0.21505821,
       -0.47573704, -0.55840755, -0.22886471,  0.5797803 ,  0.26240453,
       -0.10101186, -0.27130735,  0.3269418 , -0.3302927 ,  0.12426802,
       -0.6135263 , -0.1704403 , -0.05984049, -0.5888408 , -0.04610856,
       -0.04436302, -0.13426928, -0.1199612 ,  0.22243354,  0.19860421,
       -0.0207481 , -0.1656361 , -0.09150992,  0.01333828, -0.05937073,
       -0.0094444 , -0.08255047,  0.00510093,  0.07615712,  0.23146002,
        0.32041726,  0.27299398, -0.01216844,  0.23747851,  0.4749323 ,
        0.02114467,  0.20327869,  0.05710796,  0.0411083 , -0.12

In [17]:
def transform(word_sequence):
    return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [18]:
print(transform(model.get_nearest_neighbors(word_to_jamo('힐링'), k=10)))

[('휴식', 0.9410768747329712), ('잔잔한', 0.8843401670455933), ('휴일잔잔한', 0.8531578183174133), (':)*', 0.8489543795585632), ('</s>', 0.8407486081123352), ('§♥', 0.832419216632843), ('✨✨', 0.8287543058395386), (':)(:', 0.8189225792884827), ('(=', 0.8184536695480347), ('음악', 0.8162583708763123)]


In [19]:
print(transform(model.get_nearest_neighbors(word_to_jamo('록'), k=10)))

[('걸즈록', 0.8126246929168701), ('겸록', 0.7865683436393738), ('펑크록', 0.7669247388839722), ('그록', 0.7632830142974854), ('일렉트록', 0.7550608515739441), ('해외록', 0.7525837421417236), ('metalband', 0.7399097681045532), ('해록', 0.7382484078407288), ('록커', 0.7366763353347778), ('국내록', 0.731395959854126)]


In [20]:
print(transform(model.get_nearest_neighbors(word_to_jamo('♥♥'), k=10)))

[('㈜', 0.7310441136360168), ('♥♥♥', 0.7139729857444763), ('···', 0.690385103225708), ('^^=', 0.6812024712562561), ('●○○', 0.6719703674316406), (']..', 0.6678409576416016), ('○●', 0.6642767190933228), ('▼▶', 0.6634218692779541), ('^^~', 0.6620190739631653), ('sdf', 0.6606478095054626)]


In [21]:
print(transform(model.get_nearest_neighbors(word_to_jamo('락'), k=10)))

[('락락락', 0.805708110332489), ('legacy', 0.7924681901931763), ('rock_n_roll', 0.7869669795036316), ('락앤록', 0.7851910591125488), ('#락', 0.7833787798881531), ('rockrockrock', 0.782016932964325), ('jrock', 0.7615129351615906), ('modernrock', 0.7608477473258972), ('paintitrock', 0.7582955956459045), ('krock', 0.7581542730331421)]


In [22]:
print(transform(model.get_nearest_neighbors(word_to_jamo('기분전환'), k=10)))

[('드라이브', 0.8980928063392639), ('기분전환곡', 0.8825535774230957), ('</s>', 0.8811877965927124), ('기분전횐', 0.8726389408111572), ('기분전환과', 0.8550788760185242), ('기분전환분', 0.8546332120895386), ('시분전환', 0.8540767431259155), ('스트레스', 0.8481658697128296), ('신나는', 0.8401566743850708), ('기분전환매장음악', 0.8381237983703613)]


In [23]:
print(transform(model.get_nearest_neighbors(word_to_jamo('christmas'), k=10)))

[('#christmas', 0.9832961559295654), ('whitechristmas', 0.963708221912384), ('christmassong', 0.9538244605064392), ('christmas_carol', 0.9466356039047241), ('Christmas', 0.9173371195793152), ('chistmas', 0.9148234128952026), ('MerryChristmas', 0.9008083939552307), ('chrismas', 0.8524712920188904), ('Chirstmas', 0.8326847553253174), ('carol', 0.8305914998054504)]


In [24]:
print(transform(model.get_nearest_neighbors(word_to_jamo('radio'), k=10)))

[('radioX', 0.935064435005188), ('BBCradio1', 0.8382688164710999), ('hs', 0.8297355771064758), ('radioplay', 0.8171159625053406), ('adios', 0.7386861443519592), ('BBC', 0.7288567423820496), ('bbc', 0.7238654494285583), ('stormzy', 0.6860419511795044), ('hmltd', 0.6856263875961304), ('social', 0.6761181354522705)]


In [25]:
print(transform(model.get_nearest_neighbors(word_to_jamo('mbc'), k=10)))

[('fm', 0.942200779914856), ('15.01', 0.8820179104804993), ('14.04', 0.8812573552131653), ('15.02', 0.8799237012863159), ('20.01', 0.8785284161567688), ('mb', 0.8744148015975952), ('20.03', 0.873188316822052), ('14.12', 0.8711947798728943), ('14.05', 0.8702757954597473), ('14.01', 0.8680543899536133)]


In [26]:
print(transform(model.get_nearest_neighbors(word_to_jamo('hiphop'), k=10)))

[('khiphop', 0.9519537091255188), ('rap.hiphop', 0.9340167045593262), ('#hiphop', 0.9313274025917053), ('clubhiphop', 0.9194930195808411), ('edmhiphop', 0.9187195897102356), ('oldhiphop', 0.916515052318573), ('realhiphop', 0.8964551687240601), ('emohiphop', 0.8957729339599609), ('slowhiphop', 0.8918384313583374), ('♥ver.hiphop', 0.8837469816207886)]
